In [367]:
# Nos traemos la base de las empresas

from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
c = db.get_collection("companies")

In [554]:
# Importamos librerías

import os
import json
import requests
import pandas as pd
import pyjsonviewer
import geopandas
import shapely.geometry
from pymongo import MongoClient,GEOSPHERE
from dotenv import load_dotenv
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
load_dotenv()
clave = os.getenv(("token"))

In [571]:
# Configuramos la clave de maps

import gmaps
gmaps.configure(api_key=clave)

In [572]:
# Nos traemos la base de las compañías

from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
c = db.get_collection("companies")

In [573]:
# Y nos filtramos para buscar compañías con características similares a la nuestra
# Y observamos que existen tres ubicaciones, una en Nueva York, otra en Los Ángeles, y otra en el estado de Georgia

proy = {'_id': 0, "offices.latitude": 1,"offices.longitude": 1, "offices.address1": 1}
cond1 = {"category_code": "games_video"}
cond2 = {"founded_year": {"$gte": 2000}}
cond3 = {"number_of_employees":{"$gt": 60}}
cond4 = {"number_of_employees":{"$lt": 100}}
cond5 = {"offices.country_code": 'USA'}
list(c.find({"$and": [cond1,cond2,cond3,cond4,cond5]},proy))

[{'offices': [{'address1': '710 Ashepoint Way',
    'latitude': 17.415429,
    'longitude': 78.434422}]},
 {'offices': [{'address1': '',
    'latitude': 34.09316,
    'longitude': -118.378335}]},
 {'offices': [{'address1': '72 Madison Avenue',
    'latitude': 40.743877,
    'longitude': -73.98618}]}]

In [577]:
# Nos guardamos las coordenadas

NYCO = [40.743877, -73.98618]
LACO = [34.09316, -118.37833]
GACO = [34.17036, -84.28746]
LATNY = 40.743877
LONGNY = -73.98618
LATLA = 34.09316
LONGLA = -118.37833
LATGA = 34.17036
LONGGA = -84.28746

In [121]:
# De los distintos criterios posibles, consideremos para la ubicación de nuestra empresa la proximidad a:
# Starbucks, aeropuertos y escuelas

# asignamos keyword para buscar 
SB = "starbucks"
AP = "airport"
SC = "school"

In [124]:
radios = 500 # radio en metros para la busqueda de starbucks
radioa = 20000 # radio en metros para la busqueda de aeropuertos
radioe = 2000 # radio en metros para la busqueda de escuelas

In [277]:
# función para sacar la latitud
def latt (df_col):
    return df_col['location']['lat']

In [278]:
# función para sacar la longitud
def long (df_col):
    return df_col['location']['lng']

In [279]:
# función para sacar el typepoint
def typePoint (lng,lat):
    return {'type': 'Point', 'coordinates': [lng, lat]}

In [179]:
# Buscamos los Starbucks en NY

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATNY},{LONGNY}&radius={radios}&keyword={SB}&key={clave}"

payload={}
headers = {}

response11 = requests.request("GET", url, headers=headers, data=payload)

print(response11.text)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uEAEQTBLd6qHFbMdcqmJaPrEpkfnOkCKL6NOgNaPQUZ2pDpFd_CdOHbfrBz3LoAw-kiSb4I1jgQISkxg5K0M1ENfxVBvF9dkGvnjN9_PztHlUylNIJ7eiallRAGbZAnTFuSPCAeYYtq85v-sZv1IuHJ7yO9kGyETRcFuJrTJyiURj6We1m7ZF3Zb2lCB0QpublnzHAvUmq0SDlo6ozf-0u8fAhMtGTDKO74dDg_K5BoIu2GQgg0_ZiO60eZkQc28L8vi4b07Vh95VmAOtj9sP6KH204H3eIJub7v6M69WXmC2JGUy2MhM3TWT7lsum1P1_pCaVtTy_hD84LWQXs9JgFsTywndz_lcv5kUTPDUcUgUNiWUz6GOqovkWmlxxNGNf2SlEb20uhJNAwJs6NA7kL7qANLCgOnXFyCYWuBeHNSNnYeXGcKfY4PVB4",
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 40.745394,
               "lng" : -73.982142
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 40.74674382989271,
                  "lng" : -73.98079217010728
               },
               "southwest" : {
                  "lat" : 40.74404417010727,
                  "lng" : -73.98349182989273
     

In [321]:
# limpiamos resultados
a = response11.json()['results']
dfa = pd.DataFrame(a)
dfaa = dfa.filter(items=['name', 'geometry'])

In [322]:
# aplicamos funciones y limpiamos resultados
dfaa['latitude'] = dfaa.geometry.apply(latt)
dfaa['longitude'] = dfaa.geometry.apply(long)
dfaa["points"] = dfaa.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfaa.drop(["geometry"] , axis=1, inplace=True)
dfaa.insert(0, 'type', 'SB')

In [126]:
# Aeropuertos NY

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATNY},{LONGNY}&radius={radioa}&keyword={AP}&key={clave}"

payload={}
headers = {}

response12 = requests.request("GET", url, headers=headers, data=payload)

print(response12.text)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uEAeVkMQ-05roOEMqNWoC_wZ0CrzAWdrm7sFVRLgPwxfqm-g6NDIgQ3fMfhvSF77mqbP4adj2jbEr9ni8AajomJlMhRwMPuHxlVepdXTMZuDRlCluqkoJ6cBwKT7jtPOc2T3bSFYqz1Nwc7ojYj_SzTx3kfWbVh0yU1zRBJO0GvrAaA4gNGQ8OJjdrx_rGhGcGzmdLtXhICpN87Gq0cwbqg-PJGJv23u6TnTb_aYtpw0zmNLywF32i1RAlk1VzfshlPPTSkq4uVph7vZgA6wJ0pWC_7ZEoZ8orYs5-VlDvVr6Ephbuveauu5oypAckzHtVv_WEIKrEbEmcehyKAXlzlLT1eUCFcMD7LtMKpz2uBFhgq7uzJzeqbTXo0Iid4agdaw6ZU4_hsZtuEXsO6KQzKRn3-6kMpX53lYEOzTCyv_gvQpJ6h27J-R",
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 40.6895314,
               "lng" : -74.1744624
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 40.69546885,
                  "lng" : -74.1670314
               },
               "southwest" : {
                  "lat" : 40.68755225000001,
                  "lng" : -74.1967554
               }
         

In [326]:
# limpiamos resultados
b = response12.json()['results']
dfb = pd.DataFrame(b)
dfbb = dfb.filter(items=['name', 'geometry'])

In [327]:
# aplicamos funciones y limpiamos resultados
dfbb['latitude'] = dfbb.geometry.apply(latt)
dfbb['longitude'] = dfbb.geometry.apply(long)
dfbb["points"] = dfbb.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfbb.drop(["geometry"] , axis=1, inplace=True)
dfbb = dfbb.head(4)
dfbb.insert(0, 'type', 'Airport')

In [127]:
# Escuelas NY

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATNY},{LONGNY}&radius={radioe}&keyword={SC}&key={clave}"

payload={}
headers = {}

response13 = requests.request("GET", url, headers=headers, data=payload)

print(response13.text)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uEBGtg4jVmvNvziJlS60aEWxIX5RfsiK9UMAqb8oiaOS4xr5hMKYTJzce5fwHSv5akv1NOQqsBF_w10nbRGoTZXyi0nXOqi-JMiZC36hfS4JUhme1jJv2rOQbXjdbcodJLWeB7I-8ATk9u8Hoj5chfdS9sHJl5KPvHZKpDjYDWhv_S6h6xaY9WV_p7vPHSJayqbWywkcJWQ75UwPDEgzRsJtAQayEWR9xP0xfvWva19bfRIC_oWVHb82I1YdMuj8Pci5cBNUbVIiUf7LHVtpmJTlZdL57O-rMSmHh9pAEdNdO-Uynv6r8MuvE0Q4TzAaXMhFQs_OyWbtQx6Ykgek7BKwjb47bNnohoVn3IpMQH_Ay5KmFdtCIm8aPjE_yA0yF2LNEG-elwnNaWdgKGZDFI_XJ7ovUqir9nenaiDal_1yVtG4kuK35yM",
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 40.7354925,
               "lng" : -73.99713610000001
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 40.73694232989273,
                  "lng" : -73.99571237010727
               },
               "southwest" : {
                  "lat" : 40.73424267010729,
                  "lng" : -73.99841202989272


In [329]:
# limpiamos resultados
c = response13.json()['results']
dfc = pd.DataFrame(c)
dfcc = dfc.filter(items=['name', 'geometry'])

In [330]:
# aplicamos funciones y limpiamos resultados
dfcc['latitude'] = dfcc.geometry.apply(latt)
dfcc['longitude'] = dfcc.geometry.apply(long)
dfcc["points"] = dfcc.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfcc.drop(["geometry"] , axis=1, inplace=True)
dfcc.insert(0, 'type', 'School')

In [128]:
# Starbucks LA

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATLA},{LONGLA}&radius={radios}&keyword={SB}&key={clave}"
payload={}
headers = {}

response21 = requests.request("GET", url, headers=headers, data=payload)

print(response21.text)

{
   "html_attributions" : [],
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 34.0937659,
               "lng" : -118.3778981
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 34.09504187989273,
                  "lng" : -118.3765472201073
               },
               "southwest" : {
                  "lat" : 34.09234222010728,
                  "lng" : -118.3792468798927
               }
            }
         },
         "icon" : "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/cafe-71.png",
         "icon_background_color" : "#FF9E67",
         "icon_mask_base_uri" : "https://maps.gstatic.com/mapfiles/place_api/icons/v2/cafe_pinlet",
         "name" : "Starbucks",
         "opening_hours" : {
            "open_now" : false
         },
         "photos" : [
            {
               "height" : 1826,
               "html_at

In [338]:
# limpiamos resultados
d = response21.json()['results']
dfd = pd.DataFrame(d)
dfdd = dfd.filter(items=['name', 'geometry'])

In [339]:
# aplicamos funciones y limpiamos resultados
dfdd['latitude'] = dfdd.geometry.apply(latt)
dfdd['longitude'] = dfdd.geometry.apply(long)
dfdd["points"] = dfdd.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfdd.drop(["geometry"] , axis=1, inplace=True)
dfdd.insert(0, 'type', 'SB')

In [231]:
# Aeropuertos LA

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATLA},{LONGLA}&radius={radioa}&keyword={AP}&key={clave}"
payload={}
headers = {}

response22 = requests.request("GET", url, headers=headers, data=payload)

print(response22.text)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uEBDTLzX6IpHLHlFWFaqcGAXpA21-TUuC-FrDU4qMtkpfT5vZe6a4QKfGC_b40ov4zEkVntu46pIWx7miCrWVuT-qmBCAOwubs0v-IkGASUuP8OVUC8GTmqnq-q5YYXcB4vkJORto4UyCoUUo811z9xbRyqsS4k-Rf0JNHrajsGbKmdXIV-GeleC8BClW8TIFwItRk4qN2AKXOVndB_SqoZcr9pxLO7elEcuGrJA4xwTwoNfVoBS6vi-pc4RpSMPNbXMaQcVEQhmvtbb5PmVcDyuNIUJwrmQGSy6q4jc5o_YuLdCuqVvslyO1qzY_5bSuWgRTRO6STgYzCGQ4eUH_LlyeltaaeCWY3GPqhrwam9WuhbgSMduFdhbOIzd3QgDnoWR8GhvXVBlbnDlt2yFyGBCPBv62xp9JRwBidMg0sFZ4ta8FB2Fal4M",
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 33.9415889,
               "lng" : -118.40853
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 33.95273285,
                  "lng" : -118.384788
               },
               "southwest" : {
                  "lat" : 33.93787425,
                  "lng" : -118.416444
               }
            }
  

In [335]:
# limpiamos resultados
e = response22.json()['results']
dfe = pd.DataFrame(e)
dfee = dfe.filter(items=['name', 'geometry'])

In [336]:
# aplicamos funciones y limpiamos resultados
dfee['latitude'] = dfee.geometry.apply(latt)
dfee['longitude'] = dfee.geometry.apply(long)
dfee["points"] = dfee.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfee.drop(["geometry"] , axis=1, inplace=True)
dfee = dfee.head(6)
dfee.insert(0, 'type', 'Airport')

In [232]:
# Escuelas LA

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATLA},{LONGLA}&radius={radios}&keyword={SC}&key={clave}"

payload={}
headers = {}

response23 = requests.request("GET", url, headers=headers, data=payload)

print(response23.text)

{
   "html_attributions" : [],
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 34.0903019,
               "lng" : -118.38024
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 34.09179717989272,
                  "lng" : -118.3788913201073
               },
               "southwest" : {
                  "lat" : 34.08909752010727,
                  "lng" : -118.3815909798927
               }
            }
         },
         "icon" : "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png",
         "icon_background_color" : "#7B9EB0",
         "icon_mask_base_uri" : "https://maps.gstatic.com/mapfiles/place_api/icons/v2/school_pinlet",
         "name" : "TREE Academy",
         "opening_hours" : {
            "open_now" : false
         },
         "photos" : [
            {
               "height" : 4973,
               "ht

In [332]:
# limpiamos resultados
f = response23.json()['results']
dff = pd.DataFrame(f)
dfff = dff.filter(items=['name', 'geometry'])

In [333]:
# aplicamos funciones y limpiamos resultados
dfff['latitude'] = dfff.geometry.apply(latt)
dfff['longitude'] = dfff.geometry.apply(long)
dfff["points"] = dfff.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfff.drop(["geometry"] , axis=1, inplace=True)
dfff.insert(0, 'type', 'School')

In [334]:
dfff.head()

,type,name,latitude,longitude,points
0,School,TREE Academy,34.090302,-118.380240,"{'type': 'Point', 'coordinates': [34.0903019, ..."
1,School,Saint Victor School,34.090281,-118.380837,"{'type': 'Point', 'coordinates': [34.090281, -..."
2,School,The Graphic Design School,34.088116,-118.379454,"{'type': 'Point', 'coordinates': [34.0881162, ..."
3,School,Puppet School Los Angeles,34.088573,-118.378733,"{'type': 'Point', 'coordinates': [34.0885731, ..."
4,School,UCLA,34.092167,-118.377513,"{'type': 'Point', 'coordinates': [34.0921671, ..."


In [132]:
# Starbucks GA

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATGA},{LONGGA}&radius={radios}&keyword={SB}&key={clave}"

payload={}
headers = {}

response31 = requests.request("GET", url, headers=headers, data=payload)

print(response31.text)

{
   "html_attributions" : [],
   "results" : [],
   "status" : "ZERO_RESULTS"
}



In [133]:
# Aeropuertos GA

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATGA},{LONGGA}&radius={radioa}&keyword={AP}&key={clave}"

payload32={}
headers32 = {}

response32 = requests.request("GET", url, headers=headers32, data=payload32)

print(response32.text)

{
   "html_attributions" : [],
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "geometry" : {
            "location" : {
               "lat" : 34.3093896,
               "lng" : -84.42521889999999
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 34.31095607989272,
                  "lng" : -84.42431287010727
               },
               "southwest" : {
                  "lat" : 34.30825642010728,
                  "lng" : -84.42701252989272
               }
            }
         },
         "icon" : "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/airport-71.png",
         "icon_background_color" : "#10BDFF",
         "icon_mask_base_uri" : "https://maps.gstatic.com/mapfiles/place_api/icons/v2/airport_pinlet",
         "name" : "Cherokee County Airport",
         "photos" : [
            {
               "height" : 9000,
               "html_attributions" : [
                  "\u003ca hre

In [342]:
# limpiamos resultados
h = response32.json()['results']
dfh = pd.DataFrame(h)
dfhh = dfh.filter(items=['name', 'geometry'])

In [343]:
# aplicamos funciones y limpiamos resultados
dfhh['latitude'] = dfhh.geometry.apply(latt)
dfhh['longitude'] = dfhh.geometry.apply(long)
dfhh["points"] = dfhh.apply(lambda x: typePoint(x['latitude'], x['longitude']), axis = 1)
dfhh.drop(["geometry"] , axis=1, inplace=True)
dfhh = dfhh.head(1)
dfhh.insert(0, 'type', 'Airport')

In [344]:
dfhh

,type,name,latitude,longitude,points
0,Airport,Cherokee County Airport,34.30939,-84.425219,"{'type': 'Point', 'coordinates': [34.3093896, ..."


In [373]:
# Escuelas GA

import requests

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LATGA},{LONGGA}&radius={radioe}&keyword={SC}&key={clave}"

payload={}
headers = {}

response33 = requests.request("GET", url, headers=headers, data=payload)

print(response33.text)

# No existen ni escuelas ni Starbucks cerca de la ubicación de Georgia, por lo que la descartamos como opción

{
   "html_attributions" : [],
   "results" : [],
   "status" : "ZERO_RESULTS"
}



In [439]:
# Concatenamos las escuelas, aeropuertos, y Starbucks de NY
NY = pd.concat([dfaa, dfbb, dfcc,],ignore_index=True)

,type,name,latitude,longitude,points
0,SB,Starbucks,40.745394,-73.982142,"{'type': 'Point', 'coordinates': [40.745394, -..."
1,SB,Starbucks,40.744169,-73.983712,"{'type': 'Point', 'coordinates': [40.744169, -..."
2,SB,Starbucks,40.743839,-73.988846,"{'type': 'Point', 'coordinates': [40.743839, -..."
3,SB,Starbucks,40.748234,-73.985630,"{'type': 'Point', 'coordinates': [40.748234, -..."
4,SB,Starbucks,40.740213,-73.986946,"{'type': 'Point', 'coordinates': [40.7402132, ..."
5,SB,Starbucks,40.745007,-73.986810,"{'type': 'Point', 'coordinates': [40.7450072, ..."
6,SB,Starbucks,40.740540,-73.984221,"{'type': 'Point', 'coordinates': [40.74054, -7..."
7,SB,Starbucks,40.743592,-73.992076,"{'type': 'Point', 'coordinates': [40.743592, -..."
8,SB,Starbucks,40.748154,-73.989263,"{'type': 'Point', 'coordinates': [40.7481539, ..."
9,SB,Starbucks,40.741690,-73.990294,"{'type': 'Point', 'coordinates': [40.74169, -7..."


In [440]:
# Concatenamos las escuelas, aeropuertos, y Starbucks de LA
LA = pd.concat([dfdd, dfee, dfff,],ignore_index=True)

,type,name,latitude,longitude,points
0,SB,Starbucks,34.093766,-118.377898,"{'type': 'Point', 'coordinates': [34.0937659, ..."
1,SB,Starbucks,34.088267,-118.379060,"{'type': 'Point', 'coordinates': [34.088267, -..."
2,Airport,Los Angeles International Airport,33.941589,-118.408530,"{'type': 'Point', 'coordinates': [33.9415889, ..."
3,Airport,Van Nuys Airport,34.209640,-118.489645,"{'type': 'Point', 'coordinates': [34.2096399, ..."
4,Airport,Santa Monica Airport,34.017814,-118.447329,"{'type': 'Point', 'coordinates': [34.0178143, ..."
5,Airport,Hollywood Burbank Airport,34.198312,-118.357404,"{'type': 'Point', 'coordinates': [34.1983122, ..."
6,Airport,Whiteman Airport,34.259805,-118.411929,"{'type': 'Point', 'coordinates': [34.2598054, ..."
7,Airport,Hawthorne Municipal Airport (HHR),33.922306,-118.327103,"{'type': 'Point', 'coordinates': [33.922306, -..."
8,School,TREE Academy,34.090302,-118.380240,"{'type': 'Point', 'coordinates': [34.0903019, ..."
9,School,Saint Victor School,34.090281,-118.380837,"{'type': 'Point', 'coordinates': [34.090281, -..."


In [ ]:
# En la siguientes celdas introducimos los datos en Mongo

In [442]:
NY = geopandas.GeoDataFrame(NY, geometry=geopandas.points_from_xy(NY.longitude, NY.latitude))
LA = geopandas.GeoDataFrame(LA, geometry=geopandas.points_from_xy(LA.longitude, LA.latitude))

In [443]:
NY.drop(["points"] , axis=1, inplace=True)
LA.drop(["points"] , axis=1, inplace=True)

In [444]:
collectionNY = client.ironhack["NY"]
collectionLA = client.ironhack["LA"]

In [446]:
collectionNY.create_index([("geometry", GEOSPHERE)])
collectionLA.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [447]:
NY['geometry']=NY['geometry'].apply(lambda x:shapely.geometry.mapping(x))
LA['geometry']=LA['geometry'].apply(lambda x:shapely.geometry.mapping(x))

AttributeError: 'dict' object has no attribute '__geo_interface__'

In [448]:
NY = NY.to_dict(orient='records')
LA = LA.to_dict(orient='records')

In [449]:
collectionNY.insert_many(NY)
collectionLA.insert_many(LA)

In [451]:
# Codificamos las colecciónes
MNY = db.get_collection("NY")
MLA = db.get_collection("LA")

In [514]:
# Establecemos las coordenadas en tipo point
DNY = {'type': 'Point', 'coordinates': [-73.98618, 40.743877]}
DLA = {'type': 'Point', 'coordinates': [-118.37833, 34.09316]}

In [543]:
# Realizamos las busquedas acotando los radios
SearchNYSB = {"type" :"SB", "geometry": {"$near": {"$geometry": DNY, "$maxDistance": 250}}} #Starbuck en NY radio 250M
SearchLASB = {"type" :"SB", "geometry": {"$near": {"$geometry": DLA, "$maxDistance": 250}}} #Starbuck en LA radio 250M
SearchNYSC = {"type" :"School", "geometry": {"$near": {"$geometry": DNY, "$maxDistance": 1000}}} #Escuelas en NY radio 1000M
SearchLASC = {"type" :"School", "geometry": {"$near": {"$geometry": DLA, "$maxDistance": 1000}}} #Escuelas en LA radio 1000M
SearchNYAP = {"type" :"Airport", "geometry": {"$near": {"$geometry": DNY, "$maxDistance": 15000}}} #Aeropuertos en NY radio 15.000M
SearchLAAP = {"type" :"Airport", "geometry": {"$near": {"$geometry": DLA, "$maxDistance": 15000}}} #Aeropuertos en NLA radio 15.000M

In [548]:
# Convertimos las busquedas en listas y en data frames

NYSB = list(MNY.find(SearchNYSB))
LASB = list(MLA.find(SearchLASB))
NYSC = list(MNY.find(SearchNYSC))
LASC = list(MLA.find(SearchLASC))
NYAP = list(MNY.find(SearchNYAP))
LAAP = list(MLA.find(SearchLAAP))

DFNYSB = pd.DataFrame(NYSB)
DFLASB = pd.DataFrame(LASB)
DFNYSC = pd.DataFrame(NYSC)
DFLASC = pd.DataFrame(LASC)
DFNYAP = pd.DataFrame(NYAP)
DFLAAP = pd.DataFrame(LAAP)

In [ ]:
# Consideramos más relevantes los aeropuertos, que las escuelas y éstas que los Starbucks
# Por lo que dentro de los resultados obtenidos, multiplicamos por 3 los aeropuertos encontrados, por 2 las escuelas,
# por 1 los Starbucks, y sumamos para ver que ciudad obtiene mayor puntuación

In [574]:
PointsNY = len(NYAP)*3+len(NYSC)*2+len(NYSB)
PointsNY

16

In [575]:
PointsLA = len(LAAP)*3+len(LASC)*2+len(LASB)
PointsLA

17

In [ ]:
# Observamos que LA tiene una puntuación ligeramente superior a NY. Por lo que sería ubicación elegida

# A continuación pintamos los mapas para facilitar la visualización

In [549]:
# Juntamos los DF
MAPANY = pd.concat([DFNYSB, DFNYSC, DFNYAP],ignore_index=True)
MAPALA = pd.concat([DFLASB, DFLASC, DFLAAP],ignore_index=True)

In [595]:
# Establecemos el punto de partida de los mapas
mapNY = folium.Map(location = NYCO, zoom_start = 15)
mapLA = Map(location = LACO, zoom_start = 15)

# Añadimos la etiqueta de ubicación para las oficinas
etiquetaNY = folium.Marker(location=NYCO, tooltip = "Oficina NY")
etiquetaNY.add_to(mapNY)

etiquetaLA = folium.Marker(location=LACO, tooltip = "Oficina LA")
etiquetaLA.add_to(mapLA)

In [607]:
# Iteramos sobre el mapa de NY y le añadimos marcadores

for i,row in MAPANY.iterrows():
    distrito = {"location": [row["latitude"], row["longitude"]], "tooltip": row["type"]}
    
    if row["type"] == "SB":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="coffee",
                     icon_color="black"
        )
    elif row["type"] == "Airport":
        icono = Icon(color = "blue",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["type"] == "School":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="book",
                     icon_color="black")
        
    mark = Marker(**distrito, icon=icono)
    mark.add_to(mapNY)
mapNY

In [609]:
# Iteramos sobre el mapa de LA y le añadimos marcadores

for i,row in MAPALA.iterrows():
    distrito = {"location": [row["latitude"], row["longitude"]], "tooltip": row["type"]}
    
    if row["type"] == "SB":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="coffee",
                     icon_color="black"
        )
    elif row["type"] == "Airport":
        icono = Icon(color = "white",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["type"] == "School":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="book",
                     icon_color="black")
        
    mark = Marker(**distrito, icon=icono)
    mark.add_to(mapLA)
mapLA

In [606]:
!jupyter nbconvert --to script main.ipynb

[NbConvertApp] Converting notebook main.ipynb to script
[NbConvertApp] Writing 14322 bytes to main.py
